In [1]:
import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import torch.optim as optim
from torch.utils.data import random_split

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
device

device(type='cuda')

In [3]:

# Define dataset path
data_dir = '/kaggle/input/sign-language-detection-using-images/data'

# Define transformations for training and validation
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [4]:
dataset = datasets.ImageFolder(data_dir, transform=train_transforms)


In [5]:
num_classes = len(dataset.classes)
print(f"Number of classes: {num_classes}")
 
class_names = dataset.classes
print(f"Class names: {class_names}")


Number of classes: 35
Class names: ['1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [6]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

In [7]:
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


In [8]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Check sizes of train and validation datasets
print(f"Train set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")

Train set size: 33600
Validation set size: 8400


In [9]:
# Load the pre-trained ResNet50 model
model = models.resnet50(pretrained=True)

# Freeze the parameters of the pre-trained model (optional)
for param in model.parameters():
    param.requires_grad = False

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 150MB/s]


In [10]:
num_classes = 35
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_classes)

In [11]:
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [12]:
import sys

# Training loop
epochs = 10
best_accuracy = 0.0

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    # Training
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Print batch loss in the same line
        print(f'\rEpoch [{epoch+1}/{epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Batch Loss: {loss.item():.4f}', end='')
        sys.stdout.flush()
    
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = 100 * correct / total

    # Print epoch loss and accuracy
    print(f'\nEpoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')

    # Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader.dataset)
    val_acc = 100 * val_correct / val_total
    print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%')

    # Save the model with the best accuracy
    if val_acc > best_accuracy:
        best_accuracy = val_acc
        torch.save(model.state_dict(), 'best_resnet50_sign_language.pth')
        print(f"Model saved with accuracy: {best_accuracy:.2f}%")


Epoch [1/10], Batch [1050/1050], Batch Loss: 0.0085
Epoch 1/10, Loss: 0.1718, Accuracy: 97.92%
Validation Loss: 0.0065, Validation Accuracy: 100.00%
Model saved with accuracy: 100.00%
Epoch [2/10], Batch [1050/1050], Batch Loss: 0.0015
Epoch 2/10, Loss: 0.0063, Accuracy: 100.00%
Validation Loss: 0.0023, Validation Accuracy: 100.00%
Epoch [3/10], Batch [1050/1050], Batch Loss: 0.0020
Epoch 3/10, Loss: 0.0025, Accuracy: 99.99%
Validation Loss: 0.0020, Validation Accuracy: 99.99%
Epoch [4/10], Batch [1050/1050], Batch Loss: 0.0004
Epoch 4/10, Loss: 0.0015, Accuracy: 100.00%
Validation Loss: 0.0004, Validation Accuracy: 100.00%
Epoch [5/10], Batch [1050/1050], Batch Loss: 0.0004
Epoch 5/10, Loss: 0.0017, Accuracy: 99.97%
Validation Loss: 0.0004, Validation Accuracy: 100.00%
Epoch [6/10], Batch [1050/1050], Batch Loss: 0.0007
Epoch 6/10, Loss: 0.0023, Accuracy: 99.93%
Validation Loss: 0.0129, Validation Accuracy: 99.48%
Epoch [7/10], Batch [1050/1050], Batch Loss: 0.0001
Epoch 7/10, Loss: 0